In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint


In [2]:
# 50% fixed O&M reduction scenario
def fixed_onm_reduction_run(m):
    
    print("RUNNING 50% REDUCTION IN FIXED O&M SCENARIO")
    
    fixed_onm_variables = ["salaries_percent_FCI", "maintinance_costs_percent_FCI", "benefit_percent_of_salary", 
                "lab_fees_percent_FCI", "insurance_taxes_percent_FCI"]
        
    m = wt.watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                      scenario = scenario, source_scenario = "baseline")

    m = wt.case_study_trains.get_case_study(m=m)
        
    for fixed_onm_v in fixed_onm_variables:
        fix_value = value(getattr(m.fs.costing_param, fixed_onm_v)) * 0.5
        getattr(m.fs.costing_param, fixed_onm_v).fix(fix_value)
        
    m = run_water_tap_ro(m, source_water_category = source_water_category, 
                     desired_recovery = desired_recovery, skip_small = skip_small_base, 
                     ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = "50%_fixed_onm_reduction")
    
    return m

In [3]:
def run_water_tap_ro(m, source_water_category = None, return_df = False, 
                     skip_small = None, desired_recovery = 0, ro_bounds = None, 
                     source_scenario = None, scenario_name = None):
    
    has_ro = False
    
    ### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
    # so that the model solves and gets you results. Then runs again with set pressure.
    for key in m.fs.pfd_dict.keys():
        if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
            getattr(m.fs, key).feed.pressure.unfix()
            getattr(m.fs, key).membrane_area.unfix()
            print("Unfixing feed presure and area for", key, '...\n')
            has_ro = True
    
    wt.run_water_tap(m=m, objective=True, skip_small=skip_small)
    wt.print_ro_results(m)
    
    if has_ro is True:
        m = wt.set_bounds(m, source_water_category = ro_bounds)
        wt.print_ro_results(m)

    # If you need the system recovery to match better.... set a maximum recovery rate.
    # The previous system recovery results must be greater than the limit you set below. If this is not the case, then
    # you need to ease the bounds on the flux assumpations.
    
    if desired_recovery < 1:
        if m.fs.costing.system_recovery() > desired_recovery:
            print("Running for desired recovery -->", desired_recovery)
            m.fs.recovery_bound = Constraint(expr=m.fs.costing.system_recovery <= desired_recovery)
            m.fs.recovery_bound1 = Constraint(expr=m.fs.costing.system_recovery >= desired_recovery-1.5)
            
            if scenario_name =="baseline":
                wt.run_water_tap(m=m, objective=True, skip_small=skip_small)
            else:
                wt.run_water_tap(m=m, objective=True, skip_small=skip_small, print_model_results="summary")
            
            wt.print_ro_results(m)

        else:
            print("system recovery already lower than desired recovery. desired:", desired_recovery, 
                  "current:", m.fs.costing.system_recovery())

    
    #run model to make sure it  works
    #Readjust recovery constraint and deactivate objective constraint
#     m.recovery_bound = Constraint(expr=m.fs.costing.system_recovery >= 0)
#     m.fs.objective_function.deactivate()
    
#     print("Final check before saving RO results and/or desired recovery")
#     wt.run_water_tap(m=m, objective=False, skip_small=True)

#     wt.print_ro_results(m)
    
    if scenario_name == "baseline":
        scenario_name = m.fs.train["scenario"]
    
    if has_ro is True:
        
        #store RO variables
        ro_stash = {}
        for key in m.fs.pfd_dict.keys():
            if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                ro_stash[key] = {"feed.pressure" : getattr(m.fs, key).feed.pressure[0](),
                "membrane_area" : getattr(m.fs, key).membrane_area[0](),
                "a" : getattr(m.fs, key).a[0](),
                "b" : getattr(m.fs, key).b[0]()}

        ###### RESET BOUNDS AND DOUBLE CHECK RUN IS OK SO CAN GO INTO SENSITIVITY #####

        print("Running with unfixed, then fixed RO based on model results")
        m = wt.watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                              scenario = scenario, source_scenario = source_scenario)

        m = wt.case_study_trains.get_case_study(m=m)

        for key in m.fs.pfd_dict.keys():
            if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                getattr(m.fs, key).feed.pressure.unfix()
                getattr(m.fs, key).membrane_area.unfix()
                print("Unfixing feed presure and area for", key, '...\n')

        wt.run_water_tap(m=m, objective=True, skip_small=skip_small)

        m.fs.objective_function.deactivate()

        for key in m.fs.pfd_dict.keys():
            if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                getattr(m.fs, key).feed.pressure.fix(ro_stash[key]["feed.pressure"])
                getattr(m.fs, key).membrane_area.fix(ro_stash[key]["membrane_area"])
                getattr(m.fs, key).a.fix(ro_stash[key]["a"])
                getattr(m.fs, key).b.fix(ro_stash[key]["b"]) 

    wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)
    
    if has_ro is True: wt.print_ro_results(m)
    ############################################################
    
    # creates csv in results folder with the name: *case_study*_*scenario*.csv
    # In this case, save the final baseline result.
    
    df = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=scenario_name)
    
    if return_df is True:
        return m, df
    else:
        return m

In [4]:
df = pd.read_csv("data/baseline_cases_runs.csv")
reference = "nawi"
#scenarios = ["stage_1p5mgd"]
source_water = "seawater"    


for case_study in df[df.source_water_category == source_water].case_study.unique():
    h1 = df[df.case_study == case_study]
    for scenario in h1.scenario.unique():

    #for case_study in ["ashkelon"]:# , "tampa_bay", "carlsbad"]: #, "santa_barbara", "tampa_bay", "ashkelon"]:
    #for case_study in ["santa_barbara", "tampa_bay", "ashkelon", "carlsbad"]:

        print("------------------------------------------------------")
        print("------------------------------------------------------")
        print("-----------", case_study, scenario, "-----------------")
        print("------------------------------------------------------")
        print("------------------------------------------------------")

        desired_recovery = df[((df.case_study == case_study) & (df.scenario == scenario))].max_recovery_rate.max()

        source_water_category = df[((df.case_study == case_study) & (df.scenario == scenario))].source_water_category.max()

        skip_small_base = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_base.max()
        skip_small_sens = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_sens.max()
        ro_bounds = df[((df.case_study == case_study) & (df.scenario == scenario))].ro_bounds.max()

        m = wt.watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                              scenario = scenario, source_scenario = "baseline")

        m = wt.case_study_trains.get_case_study(m=m)

        # run and and return baseline with 0 degrees of freedom
        m = run_water_tap_ro(m, source_water_category = source_water_category, 
                             desired_recovery = desired_recovery, skip_small = skip_small_base, 
                             ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = scenario)

        #run sensitivity. then any additional model adjustment what ifs order is important
        if scenario == "baseline":
            wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                case_study = case_study, skip_small_sens = skip_small_sens)
    
            if source_water == "seawater":
                m = fixed_onm_reduction_run(m)
    

------------------------------------------------------
------------------------------------------------------
----------- ashkelon baseline -----------------
------------------------------------------------------
------------------------------------------------------
ashkelon

------- Adding Unit Processes -------
sw_onshore_intake
sulfuric_acid_addition
ferric_chloride_addition
chlorination
static_mixer
tri_media_filtration
cartridge_filtration
ro_first_pass
ro_first_stage
lime_softening
chlorination_b
caustic_soda_addition
treated_storage
backwash_solids_handling
municipal_drinking
surface_discharge
landfill
-------------------------------------

adding source

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_first_stage ...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

------------------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.745 LCOW --> 1.3049264309741502
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.76 LCOW --> 1.2791713040470285
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.775 LCOW --> 1.2544131497751505
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.79 LCOW --> 1.2305951785768883
----------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07750000000000003 LCOW --> 1.0679199161461852
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.08000000000000003 LCOW --> 1.0770834973157961
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.08250000000000003 LCOW --> 1.0863183841058366
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.085000000000


Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-20% 50.87500000000002 LCOW --> 1.3183610857299268
------- RESET -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.9721701910757412
------- Inlet Flow +-20% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 5.836425 LCOW --> 0.9815494106109146
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 6.030972500000001 LCOW --> 0.9794043099041355
-----------------------------


Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 27.0 LCOW --> 0.9141501879096217
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 28.5 LCOW --> 0.9058057221680763
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 30.0 LCOW --> 0.898418136523095
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 31.5 LCOW --> 0.891848411998951
-------------------------------------------------------------

...


WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.08797499999999997 LCOW --> 1.1520955672800928
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.09026999999999996 LCOW --> 1.1607564204900591
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.09256499999999997 LCOW --> 1.1694172737000261
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.09485999999999997 LCOW --> 1.1780781269099925
----------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 55.78286163894283 LCOW --> 1.1624441787606505
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 56.43149956497705 LCOW --> 1.1560566915915422
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 57.08013749101127 LCOW --> 1.150172039317482
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 57.72877541704549 LCOW --> 1.1447560052621575
----------------------------------------------------------------------

Degrees of Freedom: 0




WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.32000000000000006 LCOW --> 1.1233542703523915
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.3400000000000001 LCOW --> 1.1257814318727464
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.3600000000000001 LCOW --> 1.128208593393102
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.38 LCOW --> 1.130635754913457
-----------------------------

tampa_bay

------- Adding Unit Processes -------
sw_onshore_intake
sulfuric_acid_addition
ferric_chloride_addition
chlorination
static_mixer
tri_media_filtration
cartridge_filtration
ro_first_pass
ro_second_pass
lime_softening
chlorination_b
caustic_soda_addition
ammonia_addition
treated_storage
backwash_solids_handling
municipal_drinking
surface_discharge
landfill
-------------------------------------

adding source
adding splitter
params into splitter --> [0.67, 0.33]

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_second_pass ...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

-------------------------------------------------------------------

tampa_bay

------- Adding Unit Processes -------
sw_onshore_intake
sulfuric_acid_addition
ferric_chloride_addition
chlorination
static_mixer
tri_media_filtration
cartridge_filtration
ro_first_pass
lime_softening
chlorination_b
caustic_soda_addition
ammonia_addition
treated_storage
backwash_solids_handling
municipal_drinking
surface_discharge
landfill
-------------------------------------

adding source

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


SW ONSHORE INTAKE:

	total cap investment: 10.21687
	cat and chem cost: 0.0
	elec

In [27]:
#wt.display.show_train2(model_name=m)

In [35]:
#wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

In [4]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))

None
INFO: CONSTR fs.ro_first_pass.momentume_balance_eq: -0.500086603546201 =/= 0.0
INFO: CONSTR fs.ro_first_pass.pressure_waste_outlet_eq: 0.500086603546201 =/=
    0.0
INFO: CONSTR fs.ro_first_pass.permeate.eq33: -0.0003500605987803449 =/= 0.0
INFO: CONSTR fs.ro_first_stage.permeate.eq33: -5.493260147817168e-05 =/= 0.0
INFO: CONSTR fs.landfill.recovery_equation[0.0]: -1.1020654251819861e-06 =/=
    0.0
INFO: CONSTR fs.landfill.flow_balance[0.0]: 1.1884388850698369e-06 =/= 0.0
None
